In [1]:
import VASPsol as vs
import pandas as pd
import os
import numpy as np
import COSMO_TL as ctl
from dask.distributed import Client, LocalCluster, progress
import dask

In [2]:

def get_X_solute(df):
    X = df[['volume_solute', 'area_solute', 'NC_K', 'SIGMA_K','TAU', 'default_error']]
    sig_cols = [col for col in df.columns if 'sigma_solute' in col]
    sigs = df[sig_cols].to_numpy()
    X = X.to_numpy().reshape(len(df), -1)
    X = np.column_stack((X, sigs))
    return X

def get_X_solvent(df):
    X = df[['volume_solvent', 'area_solvent']]
    solvent_props_names = ['eps', 'n', 'alpha', 'beta', 'gamma', 'phi**2', 'psi**2', 'beta**2', 'NC_K','SIGMA_K','TAU', 'default_error']
    solvent_props = df[solvent_props_names].to_numpy()
    X = np.column_stack((X, solvent_props)) 
    sig_cols = [col for col in df.columns if 'sigma_solvent' in col]
    sigs = df[sig_cols].to_numpy()
    X = X.to_numpy().reshape(len(df), -1)
    X = np.column_stack((X, sigs))
    return X

def get_X(df):
    X_solute = get_X_solute(df)
    # use everything but the last 4 columns
    X_solvent = get_X_solvent(df)[:,:-4]
    X = np.column_stack((X_solute, X_solvent))
    return X
# given a dataframe return a dataframe with the mean value for all columns with error in the name
# grouped by SoluteName
def get_mean_df(df):
    df2 = df.copy()
    original_columns = list(df2.columns)
    print(original_columns)
    cols = [col for col in df2.columns if 'error' in col]
    original_columns = [col for col in original_columns if col not in cols]
    df3 = df2.groupby(['SoluteName', 'NC_K','SIGMA_K','TAU'])[cols].mean()
    df3 = df3.reset_index()
    # return a dataframe with the unique values in SoluteName and the mean values for all columns with error in the name
    # get all the other colums  from the original dataframe
    df4 = df2[original_columns]
    df4 = df4.reset_index(drop=True)
    df5 = pd.merge(df4, df3, on=['SoluteName', 'NC_K','SIGMA_K','TAU'])
    df5 = df5.drop_duplicates()
    return df5

def rejection_sampling(vector, samples=100):
    indicies = np.arange(len(vector))
    # fit a gaussian kernel to the data
    from sklearn.neighbors import KernelDensity
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(vector.reshape(-1, 1))
    # create a vector of each points probability
    logprob = kde.score_samples(vector.reshape(-1, 1))
    p = np.exp(logprob)
    # normalize the probabilities
    p = p / p.sum()

    output = []
    for sample in range(samples):
        # use the probability to sample the data
        sample = np.random.choice(indicies, p=p)
        # make sure the sample is not already in the vector
        while sample in output:
            sample = np.random.choice(indicies, p=p)
        # add the sample to the vector
        output.append(sample)
    return output

In [7]:

df = pd.read_csv('../data/vaspsol_data_3_2_2023.csv')
print(len(df))
df['error'] = df['error'].abs()
df = df[df['error'] < 10]
df = df[df['Solvent'] == 'water']
df = df[df['Charge'] == 0]
NC_K_default = 0.0025
SIGMA_K_default = 0.6
TAU_default = 0.000525
default_df = df[(df['NC_K'] == NC_K_default) & (df['SIGMA_K'] == SIGMA_K_default) & (df['TAU'] == TAU_default)]
default_df


df_to_append = default_df[['SoluteName','error']]
# rename error to default_error
df_to_append = df_to_append.rename(columns={'error': 'default_error'})
df_to_append


# match up the default error back to the original dataframe
df = pd.merge(df, df_to_append, on=['SoluteName'])
# this expanded the number of rows in the dataframe. This is not what we want
df = df.drop_duplicates('Unnamed: 0')

groups = df[df['Solvent']=='water'].groupby(['NC_K', 'SIGMA_K', 'TAU'])
# get the number of unique groups
# using the groups split of the dataframe so that unique combos of NC_K, SIGMA_K, and TAU are in each group
split = 0.99
# get the unique groups
groups = df.groupby(['NC_K', 'SIGMA_K', 'TAU'])
# get the indicies of the groups
indicies = [np.array(i) for i in groups.indices.values()]
# get the number of groups
num_groups = len(indicies)
# get the number of groups to use for training
num_train_groups = int(num_groups*split)
# get the indicies of the groups to use for training
train_indicies = np.random.choice(indicies, size=num_train_groups, replace=False)
train_indicies = np.concatenate(train_indicies.flatten())
# get the indicies of the groups to use for testing
test_indicies = np.array([i for i in np.concatenate(indicies) if i not in train_indicies])
train_df = df.iloc[train_indicies]
test_df = df.iloc[test_indicies]

X_train = get_X_solute(train_df)
X_test = get_X_solute(test_df)
y_train = train_df['error'].to_numpy()
y_test = test_df['error'].to_numpy()


# print out the shape of the training data and the training labels. Nice retro looking print statment
print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}, y_test shape: {y_test.shape}')
n_observations_train = X_train.shape[0]
n_features_train = X_train.shape[1]
n_observations_test = X_test.shape[0]
n_features_test = X_test.shape[1]

print('TRAINING SET DETAILS')
print(f'Number of observations: {n_observations_train}')
print(f'Number of features: {n_features_train}')

print('TESTING SET DETAILS')
print(f'Number of observations: {n_observations_test}')
print(f'Number of features: {n_features_test}')

6539
X_train shape: (5618, 57), y_train shape: (5618,)
X_test shape: (38, 57), y_test shape: (38,)
TRAINING SET DETAILS
Number of observations: 5618
Number of features: 57
TESTING SET DETAILS
Number of observations: 38
Number of features: 57


/tmp/ipykernel_21946/425042815.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_indicies = np.random.choice(indicies, size=num_train_groups, replace=False)


In [8]:
# lets use oversampling to balance the data
# https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html
n = 5
# get the counts for each NC_K, SIGMA_K, and TAU combo
# this is the number of observations for each group
counts = df.groupby(['NC_K', 'SIGMA_K', 'TAU']).size()
# get the max number of observations for any group
max_count = counts.max()
# reset df so that only groups with n members are included
df = df.groupby(['NC_K', 'SIGMA_K', 'TAU']).filter(lambda x: len(x) >= n)
# copy samples of the groups with less than the max number of observations
# until the number of observations for each group is equal to the max number of observations
# this will balance the data
df2 = df.groupby(['NC_K', 'SIGMA_K', 'TAU']).apply(lambda x: x.sample(max_count, replace=True))
# relabel the NC_K, SIGMA_K, and TAU columns to index_0, index_1, and index_2
# this is because the groupby function adds a new column for the groupby columns
df2 = df2.rename(columns={'NC_K': 'index_0', 'SIGMA_K': 'index_1', 'TAU': 'index_2'})
df2

Unnamed: 0 Solvent FileHandle   index_0  \
NC_K     SIGMA_K  TAU                                                       
0.000400 0.100000 0.000525 10211        1850   water    0044met  0.000400   
                           15009        2818   water    0076but  0.000400   
                           24711        4824   water    0062dio  0.000400   
                           18673        3631   water    0036tol  0.000400   
                           18673        3631   water    0036tol  0.000400   
...                                      ...     ...        ...       ...   
0.005504 0.722813 0.003820 24155        4685   water    0062dio  0.005504   
                           24155        4685   water    0062dio  0.005504   
                           8517         1608   water    0044met  0.005504   
                           24155        4685   water    0062dio  0.005504   
                           8517         1608   water    0044met  0.005504   

                                   index_1   index_2  Solvation_Energy  \
NC_K     SIGMA_K  TAU                                                    
0.000400 0.100000 0.000525 10211  0.100000  0.000525         -0.080986   
                           15009  0.100000  0.000525         -0.005511   
                           24711  0.100000  0.000525          0.041709   
                           18673  0.100000  0.000525          0.095954   
                           18673  0.100000  0.000525          0.095954   
...                                    ...       ...               ...   
0.005504 0.722813 0.003820 24155  0.722813  0.003820         -0.008190   
                           24155  0.722813  0.003820         -0.008190   
                           8517   0.722813  0.003820         -0.211970   
                           24155  0.722813  0.003820         -0.008190   
                           8517   0.722813  0.003820         -0.211970   

                                  Total_Energy   No.   SoluteName  ...  \
NC_K     SIGMA_K  TAU                                              ...   
0.000400 0.100000 0.000525 10211    -30.286090  2133     methanol  ...   
                           15009    -72.519841  2163   2-butanone  ...   
                           24711    -78.154816  2150  1,4-dioxane  ...   
                           18673    -92.563420  2126      toluene  ...   
                           18673    -92.563420  2126      toluene  ...   
...                                        ...   ...          ...  ...   
0.005504 0.722813 0.003820 24155    -78.204647  2150  1,4-dioxane  ...   
                           24155    -78.204647  2150  1,4-dioxane  ...   
                           8517     -30.417031  2133     methanol  ...   
                           24155    -78.204647  2150  1,4-dioxane  ...   
                           8517     -30.417031  2133     methanol  ...   

                                 sigma_solvent_42 sigma_solvent_43  \
NC_K     SIGMA_K  TAU                                                
0.000400 0.100000 0.000525 10211         0.080147              0.0   
                           15009         0.080147              0.0   
                           24711         0.080147              0.0   
                           18673         0.080147              0.0   
                           18673         0.080147              0.0   
...                                           ...              ...   
0.005504 0.722813 0.003820 24155         0.080147              0.0   
                           24155         0.080147              0.0   
                           8517          0.080147              0.0   
                           24155         0.080147              0.0   
                           8517          0.080147              0.0   

                                  sigma_solvent_44  sigma_solvent_45  \
NC_K     SIGMA_K  TAU                                                  
0.000400 0.100000 0.000525 10211               0.0 

In [9]:
# now grab 100 samples from each group and remake the dataframe
# this will make the data more balanced
df3 = df2.groupby(['NC_K', 'SIGMA_K', 'TAU']).apply(lambda x: x.sample(25, replace=False))
# make a normal dataframe from the groupby object
df3 = df3.reset_index(drop=True)
df3['NC_K'] = df3['index_0']
df3['SIGMA_K'] = df3['index_1']
df3['TAU'] = df3['index_2']
df3

# lets make df4 that uses rejection sampling to balance the data
# this will be used to compare the results of the oversampling
df_temp = df2
# make NC_K, SIGMA_K, and TAU columns that correspond
#df2 = df2.rename(columns={'index_0': 'NC_K', 'index_1': 'SIGMA_K', 'index_2': 'TAU'})
# extract a dataframe for each group
# break up the dataframe into multiple dataframes each of which contains only one group
# this is done by using the groupby function
# use the index of the groupby object to get the indicies of the rows in the original dataframe
# that correspond to the group

dfs = []
for i in df_temp.groupby(['NC_K', 'SIGMA_K', 'TAU']).indices.values():
    dfs.append(df_temp.iloc[i])
# for each group use rejection sampling to select 100 samples
for i in range(len(dfs)):
    indicies = rejection_sampling(dfs[i]['error'].to_numpy(), 200)
dfs = [dfs[i].iloc[indicies] for i in range(len(dfs))]
df3 = pd.concat(dfs)
df3

Unnamed: 0 Solvent FileHandle   index_0  \
NC_K     SIGMA_K  TAU                                                       
0.000400 0.100000 0.000525 15009        2818   water    0076but  0.000400   
                           15009        2818   water    0076but  0.000400   
                           24711        4824   water    0062dio  0.000400   
                           24711        4824   water    0062dio  0.000400   
                           24711        4824   water    0062dio  0.000400   
...                                      ...     ...        ...       ...   
0.005504 0.722813 0.003820 5775         1167   water    0045eth  0.005504   
                           14169        2678   water    0076but  0.005504   
                           8517         1608   water    0044met  0.005504   
                           20541        4032   water    0053phe  0.005504   
                           20541        4032   water    0053phe  0.005504   

                                   index_1   index_2  Solvation_Energy  \
NC_K     SIGMA_K  TAU                                                    
0.000400 0.100000 0.000525 15009  0.100000  0.000525         -0.005511   
                           15009  0.100000  0.000525         -0.005511   
                           24711  0.100000  0.000525          0.041709   
                           24711  0.100000  0.000525          0.041709   
                           24711  0.100000  0.000525          0.041709   
...                                    ...       ...               ...   
0.005504 0.722813 0.003820 5775   0.722813  0.003820         -0.157376   
                           14169  0.722813  0.003820         -0.017638   
                           8517   0.722813  0.003820         -0.211970   
                           20541  0.722813  0.003820         -0.193274   
                           20541  0.722813  0.003820         -0.193274   

                                  Total_Energy   No.   SoluteName  ...  \
NC_K     SIGMA_K  TAU                                              ...   
0.000400 0.100000 0.000525 15009    -72.519841  2163   2-butanone  ...   
                           15009    -72.519841  2163   2-butanone  ...   
                           24711    -78.154816  2150  1,4-dioxane  ...   
                           24711    -78.154816  2150  1,4-dioxane  ...   
                           24711    -78.154816  2150  1,4-dioxane  ...   
...                                        ...   ...          ...  ...   
0.005504 0.722813 0.003820 5775     -47.051433  2134      ethanol  ...   
                           14169    -72.531904  2163   2-butanone  ...   
                           8517     -30.417031  2133     methanol  ...   
                           20541    -82.878374  2142       phenol  ...   
                           20541    -82.878374  2142       phenol  ...   

                                 sigma_solvent_42 sigma_solvent_43  \
NC_K     SIGMA_K  TAU                                                
0.000400 0.100000 0.000525 15009         0.080147              0.0   
                           15009         0.080147              0.0   
                           24711         0.080147              0.0   
                           24711         0.080147              0.0   
                           24711         0.080147              0.0   
...                                           ...              ...   
0.005504 0.722813 0.003820 5775          0.080147              0.0   
                           14169         0.080147              0.0   
                           8517          0.080147              0.0   
                           20541         0.080147              0.0   
                           20541         0.080147              0.0   

                                  sigma_solvent_44  sigma_solvent_45  \
NC_K     SIGMA_K  TAU                                                  
0.000400 0.100000 0.000525 15009               0.0 

In [10]:
df3.to_csv('../data/vaspsol_data_3_2_2023_balanced.csv')